In [1]:
import copy
import csv
import pandas as pd

#### 1) 基準ケース作成

In [2]:
def make_standard_case(d):
    
    standard_case = {}
    
    for key in d:
        standard_case[key] = d[key]['typical']
    
    return standard_case

#### 2) 要素選択の項目のケース作成

In [3]:
def make_cases_for_options(standard_case, key, setting):
    
    cases_for_options = []
    
    options = [0]
    options.extend(setting['options'])
    options.extend([max(setting['options']) + 1])
    
    for option in options:
        case = copy.deepcopy(standard_case)
        case[key] = option
        cases_for_options.append(case)
    
    return cases_for_options

#### 3) 値入力の項目のケース作成

In [4]:
def make_cases_for_values(standard_case, key, setting):
    
    cases_for_values = []
    
    values = [
        setting['min'] - setting['dec'],
        setting['min'],
        setting['typical'],
        setting['max'],
        setting['max'] + setting['dec']
    ]

    for value in values:
        case = copy.deepcopy(standard_case)    
        case[key] = value
        cases_for_values.append(case)
    
    return cases_for_values

#### 4) ケース作成

In [5]:
def make_cases(keys_and_settings):

    # 基準ケースの作成
    standard_case = make_standard_case(keys_and_settings)

    # ケース一覧
    cases = []

    # ケースの生成
    for key in keys_and_settings:
        
        #要素選択の項目
        if keys_and_settings[key]['input_method'] == 'choose_from_options' :
            cases.extend(make_cases_for_options(standard_case, key, keys_and_settings[key]))
        
        #値入力の項目
        if keys_and_settings[key]['input_method'] == 'input_value' :
            cases.extend(make_cases_for_values(standard_case, key, keys_and_settings[key]))
        
    return cases

#### 5) csv出力

In [6]:
def output_cases(keys_and_settings):

    testcases = make_cases(keys_and_settings)
        
    cols = testcases[0].keys()

    with open("test_lv1_to_lv2_cases.csv", "w", encoding="Shift_jis") as csv_test_lv1_to_lv2_cases:
        csv_test_lv1_to_lv2_cases.write(','.join(cols) + '\n')

        for testcase in testcases:
            row = [str(testcase[col]) for col in cols]
            csv_test_lv1_to_lv2_cases.write(','.join(row) + '\n')

#### 6) 項目設定

In [7]:
keys_and_settings = {
    'region': { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4, 5, 6, 7, 8], 'typical': 1 },
    'main_occupant_room_floor_area' : { 'input_method':'input_value', 'dec':0.01, 'typical': 29.81, 'min':0.00, 'max':999.99 },
    'other_occupant_room_floor_area': { 'input_method':'input_value', 'dec':0.01, 'typical': 51.34, 'min':0.00, 'max':999.99 },
    'total_floor_area'              : { 'input_method':'input_value', 'dec':0.01, 'typical':120.08, 'min':0.00, 'max':999.99 },
    'input_method': { 'input_method':'choose_from_options', 'options': [1, 2, 3, 4], 'typical': 1 },
    'insulation_type'         : { 'input_method':'choose_from_options', 'options': [1, 2]   , 'typical': 1 },
    'insulation_type_bathroom': { 'input_method':'choose_from_options', 'options': [1, 2, 3], 'typical': 1 },
    'u_value_roof'          : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'u_value_wall'          : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'u_value_door'          : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'u_value_window'        : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'u_value_floor_bathroom': { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'u_value_floor_other'   : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'is_psi_value_base_input': { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1 },
    'psi_value_earthfloor_perimeter_entrance': { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'psi_value_earthfloor_perimeter_bathroom': { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'psi_value_earthfloor_perimeter_other'   : { 'input_method':'input_value', 'dec':0.01, 'typical': 1.00, 'min':0.00, 'max':10.00 },
    'eta_d_value_window_h': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.600, 'min':0.000, 'max':1.000 },
    'eta_d_value_window_c': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.600, 'min':0.000, 'max':1.000 },
    'is_f_value_input': { 'input_method':'choose_from_options', 'options': [1, 2], 'typical': 1 },
    'f_value_h': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.600, 'min':0.000, 'max':1.000 },
    'f_value_c': { 'input_method':'input_value', 'dec':0.001, 'typical': 0.700, 'min':0.000, 'max':1.000 }
}

output_cases(keys_and_settings)

typicalの設定  
is_f_value_inputがTrueならばf_value_hとf_value_cを入力するというような処理が入る場合  
要素選択は1,2,3,…の選択で、0と最大値+1の追加でよいか